In [65]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [66]:
df = purchase_data

In [67]:
columnName=list(df.head(0))
print(columnName)

['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']


In [68]:
df.SN.nunique() # total number of players

576

In [69]:
print(df['Item ID'].nunique()) #Unique items 

183


In [70]:
df.Price.mean()#Average price

3.050987179487176

In [71]:
df.shape[0]#the number of purchases made

780

In [72]:
df.Price.sum() #total money

2379.77

In [73]:
normed =df.groupby(['Gender', 'SN']).count().reset_index()['Gender'].value_counts(normalize=True) #male cust

In [74]:
absolute = df.groupby(['Gender', 'SN']).count().reset_index()['Gender'].value_counts(normalize=False) #female cust

In [75]:
pd.concat([normed, absolute], axis=1) #display Gender

,Gender,Gender
Male,0.840278,484
Female,0.140625,81
Other / Non-Disclosed,0.019097,11


In [76]:
gengr = df.groupby('Gender').agg(['sum', 'mean', 'count']) #gengr = gender group

level0 = gengr.columns.get_level_values(0)# get the levels of 0 
#removing the multiindex after grouping
level1 = gengr.columns.get_level_values(1)#get levels of 1
gengr.columns = level0 + '_' +level1
gengr=gengr[['Price_sum', 'Price_mean', 'Price_count']]
gengr

,Price_sum,Price_mean,Price_count
Gender,,,
Female,361.94,3.203009,113
Male,1967.64,3.017853,652
Other / Non-Disclosed,50.19,3.346000,15


In [77]:
gengr = pd.concat([gengr, absolute], axis=1, sort=True) #combine total count with unique gender count
gengr['Normalized'] = gengr.Price_sum / gengr.Gender
gengr

,Price_sum,Price_mean,Price_count,Gender,Normalized
Female,361.94,3.203009,113,81,4.468395
Male,1967.64,3.017853,652,484,4.065372
Other / Non-Disclosed,50.19,3.346000,15,11,4.562727


In [78]:
#age demographics

for_age = df[['Age', 'SN']].drop_duplicates() #this gets unique only and makes new df called for_age
#setup my bins
ages = [0, 9.90, 14.90, 19.90, 29.90, 34.90, 39.90, 99999,999999] #9.90 = less than 10 next value is 10.00
age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#assert( len(ages) > len(age_groups) ) 

df['Age_Group'] = pd.cut(df['Age'], ages, labels=age_groups)

for_age['Age_Group'] = pd.cut(for_age['Age'], ages, labels=age_groups)

age_out = pd.concat([for_age.Age_Group.value_counts(normalize=True),
                     for_age.Age_Group.value_counts()],
                    axis=1)

#age_out.to_dict()["Age_Group"]
age_out

,Age_Group,Age_Group
20-24,0.581597,335
15-19,0.185764,107
25-29,0.090278,52
30-34,0.053819,31
10-14,0.038194,22
<10,0.029514,17
35-39,0.020833,12
40+,0.000000,0


In [79]:
#age bracket, ave purchase price, total purchase value, norm totals #from original DF
age_norm = df.groupby('Age_Group').agg(['sum', 'mean', 'count'])['Price'] 
age_norm.reset_index(inplace=True)#reset the index
age_norm['unique_buyers'] = age_norm['Age_Group'].map(lambda x: age_out.to_dict()['Age_Group'].get(x))
age_norm['normed_mean'] = age_norm['sum'] /age_norm['unique_buyers'].astype('float')
age_norm

C:\Users\bryan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.


,Age_Group,sum,mean,count,unique_buyers,normed_mean
0,<10,77.13,3.353478,23,17,4.537059
1,10-14,82.78,2.956429,28,22,3.762727
2,15-19,412.89,3.035956,136,107,3.858785
3,20-24,1407.06,3.019442,466,335,4.200179
4,25-29,214.00,2.931507,73,52,4.115385
5,30-34,147.67,3.601707,41,31,4.763548
6,35-39,38.24,2.941538,13,12,3.186667
7,40+,0.00,NaN,0,0,NaN


In [80]:
df.SN.nunique(5)

576

In [81]:
df.groupby('SN')['Price'].agg(['sum', 'mean', 'count']).sort_values(by='sum', ascending=False).nlargest(5, 'sum')

,sum,mean,count
SN,,,
Lisosia93,18.96,3.792000,5
Idastidru52,15.45,3.862500,4
Chamjask73,13.83,4.610000,3
Iral74,13.62,3.405000,4
Iskadarya95,13.10,4.366667,3


In [82]:
df.groupby(df['Item Name'])['Price'].agg(['sum', 'mean', 'count']).sort_values(by='count', ascending=False).nlargest(5, 'count')

,sum,mean,count
Item Name,,,
Final Critic,59.99,4.614615,13
"Oathbreaker, Last Hope of the Breaking Storm",50.76,4.230000,12
Persuasion,28.99,3.221111,9
Nirvana,44.10,4.900000,9
"Extraction, Quickblade Of Trembling Hands",31.77,3.530000,9


In [83]:
pd.pivot_table(df, index='Item Name', columns=None, aggfunc='sum').sort_values(by='Price', ascending=False).head()

,Age,Item ID,Price,Purchase ID
Item Name,,,,
Final Critic,266,1241,59.99,6371
"Oathbreaker, Last Hope of the Breaking Storm",287,2136,50.76,4960
Nirvana,215,738,44.10,2761
Fiery Glass Crusader,187,1305,41.22,4345
Singed Scalpel,178,824,34.80,3947


Three observations that I have made from this data.
    1. per person the 30 - 34 age group spends the most.
    2. females pay more than males per person
    3. 20-24 age group is the biggest group of players.